# A Crash Course in Accelerated Database Recovery

Create a database

Create a table

Insert 1 million rows into the table

This script ran for about 30 minutes on my laptop, so...

In [91]:
USE master;

SET NOCOUNT ON;

DROP DATABASE IF EXISTS AdrDb;

CREATE DATABASE AdrDb;
GO

DROP TABLE IF EXISTS AdrTest;

USE AdrDb;

CREATE TABLE AdrTest
(RowId INT IDENTITY(1,1)
,FirstName NVARCHAR(100)
,LastName NVARCHAR(100));

DECLARE @loopcount INT, 
@looplimit INT;

SELECT @loopcount = 1, @looplimit = 250000;

WHILE @loopcount <= @looplimit
BEGIN

    INSERT INTO AdrTest
    (FirstName
    ,LastName)
    VALUES
    (N'Terence',N'Butler'),
    (N'John',N'Osbourne'),
    (N'Anthony',N'Iommi'),
    (N'William',N'Ward');

    SELECT @loopcount += 1;

END

Commands completed successfully.

: Msg 10054, Level 20, State 0, Line 8
A transport-level error has occurred when receiving results from the server. (provider: TCP Provider, error: 0 - An existing connection was forcibly closed by the remote host.)

Total execution time: 00:02:35.346

Total execution time: 00:02:35.346

# Backup Database to Default location

To prevent having to spend 30 minutes loading data, backup the database to the default backup path

For the Linux container I am running on, the default path is <span style="color: rgb(163, 21, 21); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 16px; white-space: pre;">/var/opt/mssql/data/</span>

In [ ]:
USE master
BACKUP DATABASE AdrDb TO DISK=N'/var/opt/mssql/data/AdrDb.bak';  

# Restore Database

For subsequent runs of the demo, restore the database from backup

In [ ]:
USE master;

ALTER DATABASE AdrDb
SET SINGLE_USER WITH ROLLBACK IMMEDIATE

RESTORE DATABASE AdrDb FROM DISK=N'/var/opt/mssql/data/AdrDb.bak' WITH REPLACE, STATS = 5;

# Check if Accelerated Database Recovery is Enabled

In [ ]:
USE AdrDb;

SELECT name, is_accelerated_database_recovery_on
FROM sys.databases
WHERE name = N'AdrDb';

# Delete Rows without ADR

This steo performs the following steps:

- Select and store the last log sequence for the database using fn\_dblog
- Delete 250000 rows from the table in an open transaction
- Use fn\_dblog to get a count of the delete operations from the transaction log
- Select and store the last log sequence for the database using fn\_dblog
- Rollback the delete transaction
- Use fn\_dblog to get a count of the compensation records from the transaction log

In [ ]:
USE AdrDb;

DECLARE @maxlsn NVARCHAR(46);

SELECT @maxlsn = CONCAT(N'0x',MAX([Current LSN])) FROM fn_dblog(NULL,NULL);

BEGIN TRANSACTION

DELETE FROM AdrTest
WHERE LastName = 'Osbourne';

SELECT Operation
,[Description]
,AllocUnitName
,COUNT(*) AS TranCount
FROM fn_dblog(@maxlsn,NULL)
WHERE Operation = 'LOP_DELETE_ROWS'
GROUP BY Operation, [Description],AllocUnitName;
GO

DECLARE @maxlsn NVARCHAR(46);

SELECT @maxlsn = CONCAT(N'0x',MAX([Current LSN])) FROM fn_dblog(NULL,NULL);

ROLLBACK TRANSACTION;

SELECT Operation
,[Description]
,AllocUnitName
,COUNT(*) AS TranCount
FROM fn_dblog(@maxlsn,NULL)
WHERE [Description] = 'Compensation'
GROUP BY Operation, [Description],AllocUnitName;
GO

# Enable Accelerated Database Recovery

In [ ]:
USE master;

ALTER DATABASE AdrDb
SET ACCELERATED_DATABASE_RECOVERY = ON;

SELECT name, is_accelerated_database_recovery_on
FROM sys.databases
WHERE name = N'AdrDb';

# Delete Rows with ADR

- Select and store the last log sequence for the database using fn\_dblog
- Delete 250000 rows from the table in an open transaction
- Use fn\_dblog to get a count of the delete operations from the transaction log

In [ ]:
USE AdrDb;

DECLARE @maxlsn NVARCHAR(46);

SELECT @maxlsn = CONCAT(N'0x',MAX([Current LSN])) FROM fn_dblog(NULL,NULL);

SELECT @maxlsn;

BEGIN TRANSACTION

DELETE FROM AdrTest
WHERE LastName = 'Osbourne';

-- SELECT Operation
-- ,[Description]
-- ,AllocUnitName
-- ,COUNT(*) AS TranCount
-- FROM fn_dblog(@maxlsn,NULL)
-- -- WHERE Operation = 'LOP_INSERT_ROWS'
-- GROUP BY Operation , [Description], AllocUnitName
-- ORDER BY COUNT(*) DESC;

-- SELECT  TOP 1 [Page Id] 
-- FROM fn_dblog(@maxlsn,NULL)
-- WHERE Operation = 'LOP_INSERT_ROWS'
-- AND AllocUnitName = N'sys.persistent_version_store';

SELECT Operation
,[Description]
,AllocUnitName
,COUNT(*) AS TranCount
FROM fn_dblog(@maxlsn,NULL)
WHERE Operation IN ('LOP_DELETE_ROWS', 'LOP_INSERT_ROWS')
GROUP BY Operation, [Description], AllocUnitName;
GO

# Rollback Transaction with ADR

- Get the last lsn from the transaction log using fn\_dblog
- Rollback the delete transaction
- Get all log records after the delete
- Check for compensation records

In [89]:
DECLARE @maxlsn NVARCHAR(46);

SELECT @maxlsn = CONCAT(N'0x',MAX([Current LSN])) FROM fn_dblog(NULL,NULL);

ROLLBACK TRANSACTION;

SELECT [Current LSN]
,[Transaction ID]
,[Transaction Name]
,Operation
,Context
,[Description]
,[Previous LSN]
,AllocUnitName
,[Page ID]
,[Slot ID]
,[Begin Time]
,[Database Name]
,[Number of Locks]
,[Lock Information]
,[New Split Page]
FROM fn_dblog(@maxlsn,NULL);

SELECT Operation
,[Description]
,COUNT(*) AS TranCount
FROM fn_dblog(@maxlsn,NULL)
WHERE [Description] = 'Compensation'
GROUP BY Operation, [Description];
GO

(2 rows affected)

(0 rows affected)

Total execution time: 00:00:33.200

Current LSN,Transaction ID,Transaction Name,Operation,Context,Description,Previous LSN,AllocUnitName,Page ID,Slot ID,Begin Time,Database Name,Number of Locks,Lock Information,New Split Page
0000008e:00005130:0173,0000:00041b06,NULL,LOP_DELETE_ROWS,LCX_HEAP,,0000008e:00005130:0172,dbo.AdrTest,0001:0000301a,31,NULL,NULL,1,HoBt 72057594043170816:ACQUIRE_LOCK_X OBJECT: 5:581577110:0,NULL
0000008e:00005130:0174,0000:00041b06,NULL,LOP_ABORT_XACT,LCX_CTR_ABORTED,2021/02/17 22:40:58:223,00000086:00001e38:0001,NULL,NULL,NULL,NULL,none,NULL,NULL,NULL


Operation,Description,TranCount
